# 08-02: TypeScript 映射类型 (Mapped Types)映射类型允许你基于现有类型创建新类型。

In [ ]:
// ========== 1. 基础映射类型 ==========
type User = {
  name: string;
  age: number;
  email: string;
};
// 将所有属性转为可选
type PartialUser = {
  [K in keyof User]?: User[K];
};
// 等同于: { name?: string; age?: number; email?: string; }
// 将所有属性转为只读
type ReadonlyUser = {
  readonly [K in keyof User]: User[K];
};
// 使用内置工具类型
type PartialUser2 = Partial<User>;
type ReadonlyUser2 = Readonly<User>;
type RequiredUser = Required<PartialUser>;
const user: PartialUser = { name: 'John' }; // age 和 email 可选

In [ ]:
// ========== 2. 重映射键名 ==========
// TypeScript 4.1+ 支持 as 子句重映射
type Getters<T> = {
  [K in keyof T as `get${Capitalize<string & K>}`]: () => T[K]
};
interface Person {
  name: string;
  age: number;
  location: string;
}
type PersonGetters = Getters<Person>;
// {
//   getName: () => string;
//   getAge: () => number;
//   getLocation: () => string;
// }
// 过滤特定键
type RemoveKindField<T> = {
  [K in keyof T as Exclude<K, 'kind'>]: T[K]
};
interface Circle {
  kind: 'circle';
  radius: number;
}
type KindlessCircle = RemoveKindField<Circle>;
// { radius: number }

In [ ]:
// ========== 3. 类型守卫 (Type Guards) ==========
// typeof 类型守卫
function padLeft(value: string | number, padding: string | number) {
  if (typeof padding === 'number') {
    return Array(padding + 1).join(' ') + value;
  }
  if (typeof padding === 'string') {
    return padding + value;
  }
  throw new Error(`Expected string or number, got '${padding}'.`);
}
// instanceof 类型守卫
class Bird {
  fly() { console.log('flying'); }
}
class Fish {
  swim() { console.log('swimming'); }
}
function move(animal: Bird | Fish) {
  if (animal instanceof Bird) {
    animal.fly();
  } else {
    animal.swim();
  }
}

In [ ]:
// ========== 4. 自定义类型守卫 ==========
// 使用谓词 is
interface Cat {
  name: string;
  meow(): void;
}
interface Dog {
  name: string;
  bark(): void;
}
// 自定义类型守卫函数
function isCat(animal: Cat | Dog): animal is Cat {
  return (animal as Cat).meow !== undefined;
}
function makeSound(animal: Cat | Dog) {
  if (isCat(animal)) {
    animal.meow(); // TypeScript 知道这是 Cat
  } else {
    animal.bark(); // TypeScript 知道这是 Dog
  }
}
// 更复杂的例子
type Shape =
  | { kind: 'circle'; radius: number }
  | { kind: 'square'; side: number }
  | { kind: 'rectangle'; width: number; height: number };
function isCircle(shape: Shape): shape is Extract<Shape, { kind: 'circle' }> {
  return shape.kind === 'circle';
}
function getArea(shape: Shape): number {
  if (isCircle(shape)) {
    return Math.PI * shape.radius ** 2;
  }
  // ... 其他形状
  return 0;
}

In [ ]:
// ========== 5. Branded Types ==========
// 用于创建名义类型（Nominal Typing）
type Brand<K, T> = K & { __brand: T };
// 定义 Branded Types
type UserId = Brand<string, 'UserId'>;
type PostId = Brand<string, 'PostId'>;
type Email = Brand<string, 'Email'>;
// 创建函数
function createUserId(id: string): UserId {
  return id as UserId;
}
function createPostId(id: string): PostId {
  return id as PostId;
}
function createEmail(email: string): Email {
  if (!email.includes('@')) {
    throw new Error('Invalid email');
  }
  return email as Email;
}
// 使用
const userId = createUserId('user-123');
const postId = createPostId('post-456');
const email = createEmail('test@example.com');
// 错误！不能混用
// const wrong: UserId = postId; // Error!

// 函数参数类型安全
function getUserById(id: UserId) {
  // ...
}
function getPostById(id: PostId) {
  // ...
}
getUserById(userId); // OK
// getUserById(postId); // Error! 类型不匹配
// getUserById('user-123'); // Error! 需要 UserId 类型

In [ ]:
// ========== 6. 实用映射类型 ==========
// Pick - 选取部分属性
type UserPreview = Pick<User, 'name' | 'email'>;

// Omit - 排除部分属性
type UserWithoutEmail = Omit<User, 'email'>;

// Record - 创建键值对类型
type PageInfo = Record<string, { title: string; path: string }>;
const pages: PageInfo = {
  home: { title: 'Home', path: '/' },
  about: { title: 'About', path: '/about' }
};

// Extract - 提取联合类型中的子集
type T0 = Extract<'a' | 'b' | 'c', 'a' | 'f'>; // 'a'

// Exclude - 排除联合类型中的子集
type T1 = Exclude<'a' | 'b' | 'c', 'a'>; // 'b' | 'c'

// NonNullable - 排除 null 和 undefined
type T2 = NonNullable<string | number | undefined | null>; // string | number

// Parameters - 获取函数参数类型
declare function f1(arg: { a: number; b: string }): void;
type T3 = Parameters<typeof f1>; // [{ a: number; b: string }]

// ReturnType - 获取函数返回类型
declare function f2(): { x: number; y: number };
type T4 = ReturnType<typeof f2>; // { x: number; y: number }